In [7]:
import pandas as pd


In [8]:
df_diag = pd.DataFrame()
temp = pd.read_csv('../data/DIAGNOSES_ICD.csv', iterator=True, chunksize=1000)
df_diag = pd.concat(temp, ignore_index=True)
print(df_diag.head(5))


   ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
0    1297         109   172335      1.0     40301
1    1298         109   172335      2.0       486
2    1299         109   172335      3.0     58281
3    1300         109   172335      4.0      5855
4    1301         109   172335      5.0      4254


In [10]:
df_notes = pd.DataFrame()
temp = pd.read_csv('../data/NOTEEVENTS.csv', iterator=True, chunksize=1000)
df_notes = pd.concat(temp, ignore_index=True)
print(df_notes.head(5))


   ROW_ID  SUBJECT_ID   HADM_ID   CHARTDATE CHARTTIME STORETIME  \
0     174       22532  167853.0  2151-08-04       NaN       NaN   
1     175       13702  107527.0  2118-06-14       NaN       NaN   
2     176       13702  167118.0  2119-05-25       NaN       NaN   
3     177       13702  196489.0  2124-08-18       NaN       NaN   
4     178       26880  135453.0  2162-03-25       NaN       NaN   

            CATEGORY DESCRIPTION  CGID  ISERROR  \
0  Discharge summary      Report   NaN      NaN   
1  Discharge summary      Report   NaN      NaN   
2  Discharge summary      Report   NaN      NaN   
3  Discharge summary      Report   NaN      NaN   
4  Discharge summary      Report   NaN      NaN   

                                                TEXT  
0  Admission Date:  [**2151-7-16**]       Dischar...  
1  Admission Date:  [**2118-6-2**]       Discharg...  
2  Admission Date:  [**2119-5-4**]              D...  
3  Admission Date:  [**2124-7-21**]              ...  
4  Admission Da

In [11]:
print(df_notes.shape)
print(df_diag.shape)

(2083180, 11)
(651047, 5)


In [12]:

# Unique subject IDs with Diabetes
df_sub_id_diabetes = df_diag[df_diag['ICD9_CODE'].str.startswith("250", na = False)]['SUBJECT_ID'].to_frame()

df_sub_id_diabetes = df_sub_id_diabetes.reset_index(drop=True)
df_sub_id_diabetes = df_sub_id_diabetes.drop_duplicates()
print(df_sub_id_diabetes.shape)
print(df_sub_id_diabetes.head())

(10318, 1)
   SUBJECT_ID
0         117
2         123
3         130
5         135
6         137


In [13]:
df_diag_diabetes = pd.merge(df_diag, df_sub_id_diabetes, on=['SUBJECT_ID'], how='inner')
print(df_diag_diabetes.shape)
print(df_diag_diabetes.head(2))



(215909, 5)
   ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
0    1523         117   140784      1.0      5715
1    1524         117   140784      2.0      7895


In [14]:
df_notes_diabetes = pd.merge(df_notes, df_sub_id_diabetes, on=['SUBJECT_ID'], how='inner')
print(df_notes_diabetes.shape)
print(df_notes_diabetes.head(2))

(540427, 11)
   ROW_ID  SUBJECT_ID   HADM_ID   CHARTDATE CHARTTIME STORETIME  \
0     184       28063  121936.0  2125-02-16       NaN       NaN   
1   59801       28063  121936.0  2125-02-12       NaN       NaN   

            CATEGORY DESCRIPTION  CGID  ISERROR  \
0  Discharge summary      Report   NaN      NaN   
1               Echo      Report   NaN      NaN   

                                                TEXT  
0  Admission Date:  [**2125-2-9**]              D...  
1  PATIENT/TEST INFORMATION:\nIndication: Aortic ...  


In [16]:
from string import punctuation
my_punctuation = punctuation.replace("'", "")
df_notes_diabetes['TEXT'] = df_notes_diabetes['TEXT'].str.translate(str.maketrans("", "", my_punctuation))

In [17]:
df_notes_diabetes['TEXT'].head(5)

0    Admission Date  212529              Discharge ...
1    PATIENTTEST INFORMATION\nIndication Aortic val...
2    PATIENTTEST INFORMATION\nIndication  Aortic va...
3    Sinus rhythm  Frequent atrial premature beats ...
4    Rhythm is most likely sinus rhythm with freque...
Name: TEXT, dtype: object

In [18]:
import re

df_notes_diabetes['TEXT'] = df_notes_diabetes['TEXT'].replace('[\d]', 'd',regex=True)
df_notes_diabetes['TEXT'].head(5)

0    Admission Date  dddddd              Discharge ...
1    PATIENTTEST INFORMATION\nIndication Aortic val...
2    PATIENTTEST INFORMATION\nIndication  Aortic va...
3    Sinus rhythm  Frequent atrial premature beats ...
4    Rhythm is most likely sinus rhythm with freque...
Name: TEXT, dtype: object

In [19]:
df_notes_diabetes['TEXT'] = df_notes_diabetes['TEXT'].str.lower()

df_notes_diabetes.head(5)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,184,28063,121936.0,2125-02-16,NaN,NaN,Discharge summary,Report,NaN,NaN,admission date dddddd discharge ...
1,59801,28063,121936.0,2125-02-12,NaN,NaN,Echo,Report,NaN,NaN,patienttest information\nindication aortic val...
2,59802,28063,121936.0,2125-02-11,NaN,NaN,Echo,Report,NaN,NaN,patienttest information\nindication aortic va...
3,105548,28063,121936.0,2125-02-10,NaN,NaN,ECG,Report,NaN,NaN,sinus rhythm frequent atrial premature beats ...
4,105549,28063,121936.0,2125-02-09,NaN,NaN,ECG,Report,NaN,NaN,rhythm is most likely sinus rhythm with freque...


In [22]:
df_notes_diabetes['TOKENS'] = df_notes_diabetes['TEXT'].str.split()
print(df_notes_diabetes.head(5))